# 运行、安装和使用

In [ ]:
#!docker run -p 6333:6333 -p 6334:6334 \
#     -v $(pwd)/qdrant_storage:/qdrant/storage:z \
#     qdrant/qdrant

In [ ]:
#!pip install qdrant-client[fastembed]

## Web UI

可以通过 http://127.0.0.1:6333/dashboard 访问向量库


# Quick Start

In [10]:
from qdrant_client import QdrantClient
client = QdrantClient(url='http://localhost:6333')

collections = client.get_collections()
print(collections.collections, "\n\n")
for collection in collections.collections:
    print(collection.name)

print("\n\n")
collection = client.get_collection("test")
print(collection)

[CollectionDescription(name='8b53928b64a34ebd8ebb2538324b5a76'), CollectionDescription(name='b20eb62d49bd4ee594c138339ad8cff3'), CollectionDescription(name='6e5340d16bd94445a7d4b5b942163b02'), CollectionDescription(name='test'), CollectionDescription(name='f158438fcf6e42d8b75ce8de59e5ac99'), CollectionDescription(name='ebf6f83ea303426d82589c018118ae10'), CollectionDescription(name='d455259452114442a17e746e05578106'), CollectionDescription(name='719e13cf321c4942a8a2fe3ba658ab14'), CollectionDescription(name='fba23eea8c0c4f7c87d28764851deb18')] 


8b53928b64a34ebd8ebb2538324b5a76
b20eb62d49bd4ee594c138339ad8cff3
6e5340d16bd94445a7d4b5b942163b02
test
f158438fcf6e42d8b75ce8de59e5ac99
ebf6f83ea303426d82589c018118ae10
d455259452114442a17e746e05578106
719e13cf321c4942a8a2fe3ba658ab14
fba23eea8c0c4f7c87d28764851deb18



status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=30 segments_count=8 config=Co

In [14]:
from qdrant_client.models import Distance, VectorParams
client.create_collection("test_collection",
                         vectors_config=VectorParams(size=4, distance=Distance.DOT))

True

In [13]:
# client.delete_collection("test_collection")

True

In [16]:
from qdrant_client.models import PointStruct
operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ]
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [17]:
search_result = client.query_points(collection_name="test_collection",
                                    query=[0.2, 0.1, 0.9, 0.7],
                                    with_payload=True,
                                    limit=3).points
print(search_result)

[ScoredPoint(id=4, version=0, score=1.362, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=1.273, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=1.208, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None)]


In [19]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(collection_name="test_collection",
                                    query=[0.2, 0.1, 0.9, 0.7],
                                    query_filter=Filter(
                                        must=[FieldCondition(key="city", match=MatchValue(value="London"))],
                                    )).points
print(search_result)

[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]


In [20]:
from qdrant_client.models import PointStruct
operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=7, vector=[0.19, 0.81, 0.75, 0.65], payload={"meta": {"city": "London"}}),
    ]
)
print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


In [26]:
search_result = client.query_points(collection_name="test_collection",
                                    query=[0.2, 0.6, 0.9, 0.7],
                                    with_payload=True,
                                    with_vectors=True,
                                    limit=3).points
for point in search_result:
    print(point)

id=7 version=1 score=1.654 payload={'meta': {'city': 'London'}} vector=[0.19, 0.81, 0.75, 0.65] shard_key=None order_value=None
id=1 version=0 score=1.578 payload={'city': 'Berlin'} vector=[0.05, 0.61, 0.76, 0.74] shard_key=None order_value=None
id=3 version=0 score=1.483 payload={'city': 'Moscow'} vector=[0.36, 0.55, 0.47, 0.94] shard_key=None order_value=None


In [29]:
search_result = client.query_points(collection_name="test_collection",
                                    query=[0.2, 0.6, 0.9, 0.7],
                                    with_payload=True,
                                    with_vectors=True,
                                    query_filter=Filter(
                                        must=[FieldCondition(key="meta.city", match=MatchValue(value="London"))],
                                    ),
                                    limit=3).points
for point in search_result:
    print(point)

id=7 version=1 score=1.654 payload={'meta': {'city': 'London'}} vector=[0.19, 0.81, 0.75, 0.65] shard_key=None order_value=None
